In [1]:
import json
import openai
import os
import random

import numpy as np
import pandas as pd

Import openai api key from environment variable - here I have it stored as `LOCAL_OPENAI_API_KEY` but you can just paste yours in instead

In [2]:
import os
import requests

ANTHORPIC_KEY = os.environ.get('ANTHORPIC_KEY')

def query(prompt, **kwargs):

    formattedKwargs = {}
    for key, value in kwargs.items():
        if key == 'max_tokens':
            formattedKwargs['max_tokens_to_sample'] = value
        else:
            formattedKwargs[key] = value

    payload = {
        "prompt": 'Human: ' + prompt + '\nProvide the JSON output of the code and nothing else.\n\nAssistant:',
        "model": "claude-instant-v1",
        "stop_sequences": ["\n\nHuman:"],
        "max_tokens_to_sample": 500,
        "temperature": 0,
        **formattedKwargs
    }
    headers = {
        "x-api-key": ANTHORPIC_KEY,
        "content-type": "application/json"
    }
    r = requests.post("https://api.anthropic.com/v1/complete", json=payload, headers=headers)
    r = r.json()

    return r['completion']

Great, the API key is loaded. Now we can start using the API.

In [3]:
query("q: what is 1+1?\na:")

' \n2'

This downloads the WIC dataset - you may need to install wget if you don't have it installed https://ports.macports.org/port/wget/

In [4]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-03-27 01:00:43--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 2606:50c0:8002::153, 2606:50c0:8003::153, 2606:50c0:8001::153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|2606:50c0:8002::153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip.1’

WiC_dataset.zip.1   100%[===================>] 269.52K   886KB/s    in 0.3s    

2023-03-27 01:00:44 (886 KB/s) - ‘WiC_dataset.zip.1’ saved [275984/275984]



In [5]:
import zipfile
with zipfile.ZipFile("WiC_dataset.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [6]:
train = pd.read_csv("train/train.data.txt", sep='\t', header=None)
train.columns = ["target", "pos", "position", "context-1", "context-2"]
train_gold = pd.read_csv("train/train.gold.txt", sep='\t', header=None)
train_gold.columns = ["label"]
train = pd.concat([train_gold,train], axis=1)


In [7]:
train[train.pos=="V"].tail(18)[:5]


,label,target,pos,position,context-1,context-2
5386,F,admit,V,3-1,The French doors admit onto the yard .,He admitted his errors .
5387,F,exhaust,V,0-4,Exhaust one 's savings .,This kind of work exhausts me .
5389,F,kill,V,0-2,Kill the engine .,She was killed in the collision of three cars .
5390,T,admit,V,1-1,To admit a serious thought into the mind .,She admitted us here .
5394,F,write,V,6-1,How many books did Georges Simenon write ?,Please write to me every week .


We can go and grab the last couple verbs of the train set to use as our few shots. I added some annotation for meaning

In [8]:

fewShotVerb = """ interface comparison{
    "Sense1": str, // write out the dictionary meaning of the word in the first context
    "Sense2": str, // write out the dictionary meaning of the word in the second context
    "Similar": bool, // whether the two meaings of the word are similar or used to mean different things, should be true | false
}

determineWordSimilarSense(word, context1, context2) : comparison =>{
    return ai.compare(word, context1, context2) // return the comparison object
}

//This returns the keys inside of double quotes ("KEY") so we can parse with JSON

"""

Likewise we can do it with nouns

In [9]:

fewShotNoun = """ interface comparison{
    "Sense1": str, // explain the meaning of the word in the first context so a child could understand
    "Sense2": str, // explain the meaning of the word in the second context so a child could understand
    "Similar": bool, // true | false - whether the two meaings of the word are similar or used to mean different things
}

determineWordSimilarSense(word, context1, context2) : comparison =>{
    return ai.compare(word, context1, context2) // return the comparison object
}

//This returns the keys inside of double quotes ("") so we can parse with JSON

"""

In [10]:
def testRow(row):
  pos = row[1]["pos"]
  examples = fewShotNoun if pos == "N" else fewShotVerb

  context = """determineWordSimilarSense("{}", "{}", "{}")
>>>""".format(row[1]["context-1"], row[1]["context-2"], row[1]["target"])
  res = query(examples + context)
  try:
    parsed = json.loads(res)
  except Exception as e:
    print('error parsing: ', e)
    # check if there's a true in the last 100 characters
    parsed = {
      "Context": res,
      "Similar": "true" in res.lower()[-20:]
    }

  return parsed


In [11]:
scores = {'V': 0, 'N': 0}
attempted = {'V': 0, 'N': 0}
for row in train.head(10).iterrows():
    print('scores', scores)
    print('attempted', attempted)
    actual = row[1]["label"]
    output = testRow(row)
    print('output:', output)
    pos = row[1]["pos"]
    attempted[pos] += 1

    if actual == "T":
        if output['Similar']:
            scores[pos] += 1
        
    if actual == "F":
        if not output['Similar']:
            scores[pos] += 1



scores {'V': 0, 'N': 0}
attempted {'V': 0, 'N': 0}
output: {'Sense1': 'transport or convey (people or goods) from one place to another', 'Sense2': 'continue to have or retain (an idea, feeling, or condition)', 'Similar': False}
scores {'V': 1, 'N': 0}
attempted {'V': 1, 'N': 0}
output: {'Sense1': 'move or proceed in a particular direction or course', 'Sense2': 'serve as a means of communication or transmission between', 'Similar': False}
scores {'V': 2, 'N': 0}
attempted {'V': 2, 'N': 0}
output: {'Sense1': 'destroy or damage something by force', 'Sense2': 'interrupt the continuity or course of; make a break in', 'Similar': False}
scores {'V': 3, 'N': 0}
attempted {'V': 3, 'N': 0}
output: {'Sense1': 'A protective garment worn by athletes to support the genitals. ', 'Sense2': 'A container used by bees to store honey. ', 'Similar': False}
scores {'V': 3, 'N': 0}
attempted {'V': 3, 'N': 1}
output: {'Sense1': 'A prestigious institution dedicated to the arts, sciences, and humanities. ', 'Se

In [12]:
scores, attempted

({'V': 7, 'N': 0}, {'V': 7, 'N': 3})

In [13]:
dev = pd.read_csv("dev/dev.data.txt", sep='\t', header=None)
dev.columns = ["target", "pos", "position", "context-1", "context-2"]
dev_gold = pd.read_csv("dev/dev.gold.txt", sep='\t', header=None)
dev_gold.columns = ["label"]
dev = pd.concat([dev_gold,dev], axis=1)


In [14]:
devResults = {}
complete = 0
correct = 0

In [15]:

for row in dev.iterrows():

    if row[0] in devResults:
        continue

    q1 = row[1]["context-1"]
    q2 = row[1]["context-2"]
    target = row[1]["target"]
    actual = row[1]["label"]
    
    pos = row[1]["pos"]

    output = testRow(row)
    
    myResults = {}
    myResults["q1"] = q1
    myResults["q2"] = q2

    myResults["pos"] = row[1]["pos"]

    myResults["target"] = target

    myResults["output"] = output

    myResults["actual"] = actual
    devResults[row[0]] = myResults
    complete +=1
    try:
        if actual == "T":
            if output['Similar']:
                correct += 1
        if actual == "F":
            if not output['Similar']:
                correct += 1
    except:
        print('output', output)

    pct = correct/complete
    pct = round(pct, 2)

    print ("Score: {}, Complete: {} Correct: {} Wrong: {}".format(pct, complete, correct, complete-correct))
    with open('claude-instant-v1-code-0shot.json', 'w') as f:
        json.dump(devResults, f)


Score: 1.0, Complete: 1 Correct: 1 Wrong: 0
Score: 1.0, Complete: 2 Correct: 2 Wrong: 0
Score: 0.67, Complete: 3 Correct: 2 Wrong: 1
Score: 0.75, Complete: 4 Correct: 3 Wrong: 1
Score: 0.8, Complete: 5 Correct: 4 Wrong: 1
Score: 0.67, Complete: 6 Correct: 4 Wrong: 2
Score: 0.71, Complete: 7 Correct: 5 Wrong: 2
Score: 0.75, Complete: 8 Correct: 6 Wrong: 2
Score: 0.78, Complete: 9 Correct: 7 Wrong: 2
Score: 0.8, Complete: 10 Correct: 8 Wrong: 2
Score: 0.82, Complete: 11 Correct: 9 Wrong: 2
Score: 0.83, Complete: 12 Correct: 10 Wrong: 2
Score: 0.85, Complete: 13 Correct: 11 Wrong: 2
Score: 0.86, Complete: 14 Correct: 12 Wrong: 2
Score: 0.87, Complete: 15 Correct: 13 Wrong: 2
Score: 0.81, Complete: 16 Correct: 13 Wrong: 3
Score: 0.82, Complete: 17 Correct: 14 Wrong: 3
Score: 0.78, Complete: 18 Correct: 14 Wrong: 4
Score: 0.79, Complete: 19 Correct: 15 Wrong: 4
Score: 0.8, Complete: 20 Correct: 16 Wrong: 4
Score: 0.81, Complete: 21 Correct: 17 Wrong: 4
Score: 0.82, Complete: 22 Correct: 18 